<a href="https://colab.research.google.com/github/PhilipRuebeling/master_thesis/blob/main/photonic_devices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}\newcommand{\bra}[1]{\left\langle
{#1}\right|}$$

In [ ]:
%pip install fst-pso  # particle swarm optimization

In [ ]:
import numpy as np
import fstpso as fp

import scipy.special

# Quantum Frequency Processor (QFP) with coherent states

We can use coherent states to operate our QFP. This can be easily achieved by noting, that for a coherent state $\ket{\alpha} = D(\alpha)\ket{vac}$, where the displacement operator is defined by $D(\alpha) = \exp (\sum_{r} \alpha_{r} a^{\dagger}_{r} + \alpha_{r}^{*} a_{r})$. When we consider the mode transformation

$a^{\dagger}_{r} \rightarrow Sa^{\dagger}_{r}S^{\dagger} = \sum_{q} (-ie^{-i\vartheta})^{q} J_{q}(m)a^{\dagger}_{r+q}$.

parameters of the QFP

In [ ]:
n_layers, n_bins = 4, 32  # topological parameters of the QFP

In [ ]:
def unitaryV(parameters_ps, parameters_em) :
    V = np.eye(bins, dtype='complex64')
    for layer in range(0, n_layers) :
        U_em = em.unitaryFD(parameters_em[layer], mod_depth, n_bins)
        U_ps = ps.unitaryFD(parameters_ps[layer], n_bins)
        V = V @ U_ps @ U_eom
    return V

In [ ]:
def optimize_circuit_fuzzypso() :
  FP = fp.FuzzyPSO()
  search_space = fp.setupSearchSpace(a_min, a_max)
  FP.set_search_space(search_space)
  FP.set_fitness(costfunction)  
  FP.set_swarm_size(swarm_size)
  best_solution, fitness = FP.solve_with_fstpso(max_iter=iterations)
  return best_solution, fitness

# Electro-optic modulator (EOM)

The phase modulation of an [EOM](https://en.wikipedia.org/wiki/Electro-optic_modulator) is described by its driving voltage

$a(t) = \sum_{k=1}^{m} a_{k}\sin(\Omega_{k} t + \varphi_{k})$

Here we call $m = 1, 2, \dots$ the modulation depth. When we concentrate on single-tone modulation we will set this to be equal to $1$.


In [ ]:
mod_depth = 1     # modulation depth
a_min = 00.00     # minimal modulation amplitude
a_max = 10.00     # maximal modulation amplitude

This defines the unitary in time domain (TD).

In [ ]:
def unitaryTD(parameters, mod_depth, n_bins) :
    U = np.zeros(shape=(n_bins, n_bins), dtype="complex64")
    amplitude, phase = parameters
    for i in range(0, d):
        U[i][i] += np.exp(1j*(amplitude*np.sin(np.pi*i/d + phase)))
    return U

In [ ]:
# testing if unitaryTD returns a unitary matrix
U = np.around(unitaryTD(parameters, mod_depth, n_bins), decimals=9)
is_unitary = np.allclose(np.eye(n_bins), U @ RT.adjoint(U))
print('unitaryTD() returns a unitary matrix: ' + str(is_unitary))

Plotting the modulation ignal

In [ ]:
def plot_modulation(parameters, mod_depth, n_bins) :
    amplitude, phase = parameters
    t = np.arange(0, 10*n_bins)
    s = np.around(amplitude*np.sin(np.pi*t/n_bins + phase), decimals=9)
    plt.title('modulation signal ot the EOM in TD')
    plt.plot(t/n_bins, s)
    plt.xlabel('time [norm.]')
    plt.ylabel('driving voltage [arb.]')
    plt.show()
    plt.close()
    pass

In [ ]:
plot_modulation(parameters, mod_depth, n_bins)

Here is the corresponding unitary in frequency domain (FD).

In [ ]:
def unitaryFD(parameters, mod_depth, n_bins) :
    F = dft(bins)/np.sqrt(bins)
    U = unitaryTD(parameters, mod_depth, d) #doesn't involve yet.
    return F @ U @ RT.adjoint(F)

In [ ]:
def besselFD(parameters, n_bins, f=0.42) :
    U = np.zeros(shape=(d,d), dtype='complex64')
    amplitude, phase = parameters
    for i in range(0, d) :
        for j in range(0, d) :
            if(np.abs(i-j) <= math.ceil(amplitude+2)) :
                U[i][j] = special.jv(i-j, f*amplitude)*(-1j*np.exp(-1j*phase))**(i-j)
    return U

# Phase Shaper (PS)



In [1]:
def unitaryFD(parameters, n_bins) :
    U = np.array(np.zeros(shape=(n_bins, n_bins)), dtype='complex64')
    for i in range(0, n_bins):
        U[i][i] = np.exp(1j*theta[i])
    return U

# BAS22